## Business Understanding

## Imports

In [1]:
import pandas as pd
import re
import webbrowser
import scrapy

In [2]:
df =  pd.read_csv('Datafiniti_Hotel_Reviews.csv')

In [3]:
pd.set_option('display.max_rows', 0)
pd.set_option('display.max_columns', 0)

## Data Exploration

In [4]:
df.shape

(10000, 25)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 25 columns):
id                      10000 non-null object
dateAdded               10000 non-null object
dateUpdated             10000 non-null object
address                 10000 non-null object
categories              10000 non-null object
primaryCategories       10000 non-null object
city                    10000 non-null object
country                 10000 non-null object
keys                    10000 non-null object
latitude                10000 non-null float64
longitude               10000 non-null float64
name                    10000 non-null object
postalCode              10000 non-null object
province                10000 non-null object
reviews.date            10000 non-null object
reviews.dateSeen        10000 non-null object
reviews.rating          10000 non-null float64
reviews.sourceURLs      10000 non-null object
reviews.text            9999 non-null object
reviews.title   

#### Initial Dataset Feature Descriptions

- 'id' = id of review <br>
- 'dateAdded' = date added string (month and year) <br>
- 'dateUpdated' = date updated string (month and year)  <br>
- 'address' = address of hotel <br>
- 'categories' = type of service provided <br>
- 'primaryCategories' = primary service <br>
- 'city' = city of hotel <br>
- 'country' = country of hotel (only US) <br>
- 'keys' = ????looks like URL extension / API key???? <br>
- 'latitude' = hotel location latitude <br>
- 'longitude' = hotel location longitude <br>
- 'name' =  name of hotel <br>
- 'postalCode' = postcode of hotel <br> 
- 'province' = state that hotel is located <br>
- 'reviews.date' = ??? <br>
- 'reviews.dateSeen' = ??? <br>
- 'reviews.rating' = score rating 1-5 <br>
- 'reviews.sourceURLs' = tripadvisor description of hotel <br>
- 'reviews.text' = written review <br>
- 'reviews.title' = title of review <br>
- 'reviews.userCity' = city of user who gave a review <br>
- 'reviews.userProvince' = state the user is located <br>
- 'reviews.username' =  username of reviewer <br>
- 'sourceURLs' = URL of the review <br>
- 'websites' = website of the hotel <br>

In [6]:
df.head()

,id,dateAdded,dateUpdated,address,categories,primaryCategories,city,country,keys,latitude,longitude,name,postalCode,province,reviews.date,reviews.dateSeen,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.userCity,reviews.userProvince,reviews.username,sourceURLs,websites
0,AVwc252WIN2L1WUfpqLP,2016-10-30T21:42:42Z,2018-09-10T21:06:27Z,5921 Valencia Cir,"Hotels,Hotels and motels,Hotel and motel reser...",Accommodation & Food Services,Rancho Santa Fe,US,us/ca/ranchosantafe/5921valenciacir/359754519,32.990959,-117.186136,Rancho Valencia Resort Spa,92067,CA,2013-11-14T00:00:00Z,"2016-08-03T00:00:00Z,2016-07-26T00:00:00Z,2016...",5.0,https://www.hotels.com/hotel/125419/reviews%20/,Our experience at Rancho Valencia was absolute...,Best romantic vacation ever!!!!,NaN,NaN,Paula,http://www.hotels.com/ho125419/%25252525253Flo...,http://www.ranchovalencia.com
1,AVwc252WIN2L1WUfpqLP,2016-10-30T21:42:42Z,2018-09-10T21:06:27Z,5921 Valencia Cir,"Hotels,Hotels and motels,Hotel and motel reser...",Accommodation & Food Services,Rancho Santa Fe,US,us/ca/ranchosantafe/5921valenciacir/359754519,32.990959,-117.186136,Rancho Valencia Resort Spa,92067,CA,2014-07-06T00:00:00Z,"2016-08-02T00:00:00Z,2016-08-26T00:00:00Z,2016...",5.0,https://www.hotels.com/hotel/125419/reviews%20/,Amazing place. Everyone was extremely warm and...,Sweet sweet serenity,NaN,NaN,D,http://www.hotels.com/ho125419/%25252525253Flo...,http://www.ranchovalencia.com
2,AVwc252WIN2L1WUfpqLP,2016-10-30T21:42:42Z,2018-09-10T21:06:27Z,5921 Valencia Cir,"Hotels,Hotels and motels,Hotel and motel reser...",Accommodation & Food Services,Rancho Santa Fe,US,us/ca/ranchosantafe/5921valenciacir/359754519,32.990959,-117.186136,Rancho Valencia Resort Spa,92067,CA,2015-01-02T00:00:00Z,"2016-11-15T00:00:00Z,2016-08-23T00:00:00Z,2016...",5.0,https://www.hotels.com/hotel/125419/reviews%20/,We booked a 3 night stay at Rancho Valencia to...,Amazing Property and Experience,NaN,NaN,Ron,http://www.hotels.com/ho125419/%25252525253Flo...,http://www.ranchovalencia.com
3,AVwdOclqIN2L1WUfti38,2015-11-28T19:19:35Z,2018-09-10T21:06:16Z,7520 Teague Rd,"Hotels,Hotels and motels,Travel agencies and b...",Accommodation & Food Services,Hanover,US,us/md/hanover/7520teaguerd/-2043779672,39.155929,-76.716341,Aloft Arundel Mills,21076,MD,2016-05-15T00:00:00Z,"2016-05-21T00:00:00Z,2016-07-31T00:00:00Z",2.0,https://www.tripadvisor.com/Hotel_Review-g4118...,Currently in bed writing this for the past hr ...,"Never again...beware, if you want sleep.",Richmond,VA,jaeem2016,http://www.yellowbook.com/profile/aloft-arunde...,http://www.starwoodhotels.com/alofthotels/prop...
4,AVwdOclqIN2L1WUfti38,2015-11-28T19:19:35Z,2018-09-10T21:06:16Z,7520 Teague Rd,"Hotels,Hotels and motels,Travel agencies and b...",Accommodation & Food Services,Hanover,US,us/md/hanover/7520teaguerd/-2043779672,39.155929,-76.716341,Aloft Arundel Mills,21076,MD,2016-07-09T00:00:00Z,2016-07-31T00:00:00Z,5.0,https://www.tripadvisor.com/Hotel_Review-g4118...,I live in Md and the Aloft is my Home away fro...,ALWAYS GREAT STAY...,Laurel,MD,MamaNiaOne,http://www.yellowbook.com/profile/aloft-arunde...,http://www.starwoodhotels.com/alofthotels/prop...


#### Removing Nulls

In [7]:
df.dropna(axis=0,subset=['reviews.text'],inplace=True)

In [8]:
df.shape

(9999, 25)

#### Removing Unnecessary Columns

In [9]:
df.drop(axis=1,columns=['id','country','reviews.date','reviews.dateSeen','reviews.title','reviews.userCity','reviews.userProvince','reviews.username'],inplace=True)

In [10]:
df.shape

(9999, 17)

## Feature Engineering

### Extract Features Via Webscraping Using URLs From Dataset

#### Change Reviews With Multiple Review Source URLs to a Singular Source

In [11]:
pd.set_option('display.max_colwidth', 280)

In [12]:
df['reviews.sourceURLs'].head(20)

0                                                                                                                                                                                                                                             https://www.hotels.com/hotel/125419/reviews%20/
1                                                                                                                                                                                                                                             https://www.hotels.com/hotel/125419/reviews%20/
2                                                                                                                                                                                                                                             https://www.hotels.com/hotel/125419/reviews%20/
3                                                https://www.tripadvisor.com/Hotel_Review-g41185-d1239660-Reviews-Aloft_Arundel_Mills-Hanover_

In [13]:
df[df['reviews.sourceURLs'].str.contains(',')].shape

(1398, 17)

In [14]:
df[df['sourceURLs'].str.contains(',')].shape

(9999, 17)

In [15]:
df['reviews.sourceURLs'] = df['reviews.sourceURLs'].apply(lambda x: x.split(',')[0])

In [16]:
df['sourceURLs'] = df['sourceURLs'].apply(lambda x: x.split(',')[0])

In [17]:
df['reviews.sourceURLs'].head(20)

0                                                                                                             https://www.hotels.com/hotel/125419/reviews%20/
1                                                                                                             https://www.hotels.com/hotel/125419/reviews%20/
2                                                                                                             https://www.hotels.com/hotel/125419/reviews%20/
3                                                  https://www.tripadvisor.com/Hotel_Review-g41185-d1239660-Reviews-Aloft_Arundel_Mills-Hanover_Maryland.html
4                           https://www.tripadvisor.com/Hotel_Review-g41185-d1239660-Reviews-or10-Aloft_Arundel_Mills-Hanover_Maryland.html%2525252523REVIEWS
5                           https://www.tripadvisor.com/Hotel_Review-g41185-d1239660-Reviews-or20-Aloft_Arundel_Mills-Hanover_Maryland.html%2525252523REVIEWS
6                                                  h

In [18]:
df[df['reviews.sourceURLs'].str.contains(',')].shape

(0, 17)

In [19]:
df['sourceURLs'].head(20)

0                   http://www.hotels.com/ho125419/%25252525253Flocale%25252525253Den_US%252525252526pos%25252525253DHCOM_US
1                   http://www.hotels.com/ho125419/%25252525253Flocale%25252525253Den_US%252525252526pos%25252525253DHCOM_US
2                   http://www.hotels.com/ho125419/%25252525253Flocale%25252525253Den_US%252525252526pos%25252525253DHCOM_US
3                                                      http://www.yellowbook.com/profile/aloft-arundel-mills_1896561264.html
4                                                      http://www.yellowbook.com/profile/aloft-arundel-mills_1896561264.html
5                                                      http://www.yellowbook.com/profile/aloft-arundel-mills_1896561264.html
6                                                      http://www.yellowbook.com/profile/aloft-arundel-mills_1896561264.html
7                                                      http://www.yellowbook.com/profile/aloft-arundel-mills_1896561264.html


In [20]:
df[df['sourceURLs'].str.contains(',')].shape

(0, 17)

#### Creating Columns With URL Roots

In [21]:
pattern = re.compile(r'https?://(www\.)?(\w+)(\.\w+)')

In [22]:
df.loc[:,'review_source_URL_root'] = df['reviews.sourceURLs'].apply(lambda x: pattern.findall(x)[0][1])

In [23]:
df.loc[:,'URL_root'] = df['sourceURLs'].apply(lambda x: pattern.findall(x)[0][1])

In [24]:
df.groupby('review_source_URL_root').size().reset_index(name='count').sort_values(by='count',ascending=False)

,review_source_URL_root,count
8,tripadvisor,5695
3,expedia,1830
0,booking,1222
4,hotels,1190
1,citysearch,21
5,insiderpages,15
10,yellowpages,13
7,theknot,9
6,marriott,2
2,dexknows,1


In [25]:
df.groupby('URL_root').size().reset_index(name='count').sort_values(by='count',ascending=False)

,URL_root,count
4,foursquare,2026
19,yellowpages,1719
16,tripadvisor,1680
3,expedia,1531
1,booking,1054
18,yellowbook,1043
6,hotels,278
2,citysearch,243
12,telegraph,93
17,weddingwire,57


In [26]:
df[['reviews.sourceURLs','review_source_URL_root','sourceURLs','URL_root']].head()

,reviews.sourceURLs,review_source_URL_root,sourceURLs,URL_root
0,https://www.hotels.com/hotel/125419/reviews%20/,hotels,http://www.hotels.com/ho125419/%25252525253Flocale%25252525253Den_US%252525252526pos%25252525253DHCOM_US,hotels
1,https://www.hotels.com/hotel/125419/reviews%20/,hotels,http://www.hotels.com/ho125419/%25252525253Flocale%25252525253Den_US%252525252526pos%25252525253DHCOM_US,hotels
2,https://www.hotels.com/hotel/125419/reviews%20/,hotels,http://www.hotels.com/ho125419/%25252525253Flocale%25252525253Den_US%252525252526pos%25252525253DHCOM_US,hotels
3,https://www.tripadvisor.com/Hotel_Review-g41185-d1239660-Reviews-Aloft_Arundel_Mills-Hanover_Maryland.html,tripadvisor,http://www.yellowbook.com/profile/aloft-arundel-mills_1896561264.html,yellowbook
4,https://www.tripadvisor.com/Hotel_Review-g41185-d1239660-Reviews-or10-Aloft_Arundel_Mills-Hanover_Maryland.html%2525252523REVIEWS,tripadvisor,http://www.yellowbook.com/profile/aloft-arundel-mills_1896561264.html,yellowbook


#### Tripadvisor.com

Should be easy to scrape average scores for subcategories. <br>

Review round-ups found in 'reviews.sourceURLs' not 'sourceURLs'.

Categories: 

- Location 
- Cleanliness 
- Service  
- Value 

In [27]:
tripadvisor_df = df[df['review_source_URL_root']=='tripadvisor']['reviews.sourceURLs'].reset_index(drop=True)

In [28]:
tripadvisor_df.shape

(5695,)

In [29]:
# for i in range(5640,5662):
#     webbrowser.open(tripadvisor_df.loc[i], new=2)

#### Hotels.com

Not easy to scrape and majority of URLs led to page not found.

In [30]:
hotels_df = df[df['URL_root']=='hotels']['sourceURLs'].reset_index(drop=True)

In [31]:
hotels_df.shape

(278,)

In [32]:
# for i in range(0,20):
#     webbrowser.open(hotels_df.loc[i], new=2)

#### Expedia.com

Should be easy to scrape average scores for subcategories. <br>

Review round-ups found in 'reviews.sourceURLs' not 'sourceURLs'.

Categories: 

- Room cleanliness
- Service & staff
- Room comfort
- Hotel condition

In [33]:
expedia_df = df[df['URL_root']=='expedia']['reviews.sourceURLs'].reset_index(drop=True)

In [34]:
expedia_df.shape

(1531,)

In [35]:
# for i in range(120,140):
#     webbrowser.open(expedia_df.loc[i], new=2)

#### Booking.com

Majority of booking.com URLs led to page not found. 

In [36]:
booking_df = df[df['URL_root']=='booking']['sourceURLs'].reset_index(drop=True)

In [37]:
booking_df.shape

(1054,)

In [38]:
# for i in range(86,106):
#     webbrowser.open(booking_df.loc[i], new=2)

#### Foursquare.com

Foursquare is for bar menus and do not contain reviews. <br> 

Menus found in 'sourceURLs'.

In [39]:
foursquare_df = df[df['URL_root']=='foursquare']['sourceURLs'].reset_index(drop=True)

In [40]:
foursquare_df.shape

(2026,)

In [41]:
# for i in range(86,106):
#     webbrowser.open(foursquare_df.loc[i], new=2)

#### Yellowpages.com

The majority of the URLs led to page not found or pages that could not be scraped.

In [42]:
yellowpages_df =  df[df['URL_root']=='yellowpages']['sourceURLs'].reset_index(drop=True)

In [43]:
yellowpages_df.shape

(1719,)

In [44]:
# for i in range(86,106):
#     webbrowser.open(yellowpages_df.loc[i], new=2)

#### Assessment of Websites

- Tripadvisor and Expedia are the two most reliable sites to scrape more data on hotel review scores

- They both have two very similar sub-categories; Cleanliness and service

- In total there are 7226 reviews from our original dataset that are from expedia.com and tripadvisor.com

### Webscraping

In [46]:
df['reviews.rating'].unique()

array([5.  , 2.  , 4.  , 3.  , 1.  , 4.8 , 4.6 , 3.55, 4.4 , 4.15, 2.5 ,
       3.95, 2.9 , 3.35, 3.75, 4.5 , 2.1 , 1.65, 3.15, 2.7 , 1.45, 2.75,
       2.3 , 3.5 , 4.25, 1.25, 1.9 , 3.45, 3.25, 4.75])

In [47]:
df.groupby('reviews.rating').size().reset_index(name='count').sort_values(by='count',ascending=False)

,reviews.rating,count
29,5.00,4384
21,4.00,2397
12,3.00,1186
0,1.00,572
5,2.00,561
28,4.80,162
26,4.60,155
24,4.40,102
19,3.75,91
22,4.15,78


#### Tripadvisor